# <font color=black> Fig 04b Brain and spinal cord functional connectivity </font>
<hr style="border:1px solid black">

### Imports

In [ ]:
#-------- Basics
import sys,json,os, glob, re
import pandas as pd
import numpy as np

#------ load config file
main_dir='/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/2025_brsc_aging_project/'
sys.path.append(main_dir+'/code/')
config_file=main_dir + '/config/analyses/brsc_functional.json'
with open(config_file) as config_f:
    config = json.load(config_f) # load config info
output_dir=config['project_dir'] + config["seed2seed"]['analysis_dir']['brain'] 


#-------- Plotting
#import seaborn as sns
#import statsmodels.api as sm
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from plotting import Plotting
plot=Plotting(config_file,output_dir)
colors_fc = ["#F2ECDF","#D5E5D7","#7BCEBC","#40BFAA","#04AF98","#029E89","#008C79"]  # green color map
fc_cmap = LinearSegmentedColormap.from_list("my_colormap", colors_fc) # create the custom fc cmap

#-------- Statistics
from brsc_statistics import Statistics 
from statsmodels.stats.multitest import multipletests
stat_func=Statistics(config=config,ana_dir="",analysis="")

#------- FC analyses
from connectivity.seed_to_target import Seed2target
seed2seed=Seed2target(config_file,IDs=config["participants_IDs_ALL"],kind="corr",seed_kind="atlas",structure=["brain"]) # initialize the function

%matplotlib inline
%load_ext autoreload
%autoreload 2


<hr style="border:1px solid black">

### <font color=#04AF98> A Brain global functional connectivity 
#### <font color=#04AF98> A.1 Timeseries extraction

In [ ]:
#timeseries_labels,labels_list=seed2seed.extract_atlas_data(rmv_col="remove",standardize='zscore',redo=True,n_jobs=3)
ts,timeseries_labels,labels_list=seed2seed.extract_atlas_data(standardize='zscore',redo=False,n_jobs=3)



#### <font color=#04AF98> A.2 Correlation analysis

In [ ]:
timeseries_zmean=[];labels_concat=[]; results = [];

#---- Correlation
corr_concat_df,corr_half_concat_df,corr_mean_df =seed2seed.correlation_df(timeseries_labels,
                                                                          timeseries_labels,
                                                                          labels_list=labels_list,
                                                                          tag= "_Schaefer200Cobra_z",
                                                                          groups=["ALL"],#,'YA','MA','OA'],
                                                         labels=True,
                                                      #partial_ts=timeseries_csf,
                                                         redo=False, n_jobs=6)


In [ ]:

#---- Create sub df including sensorimotor region only
df_SomMot = corr_mean_df["ALL"][corr_mean_df["ALL"]['seed1'].str.contains("SomMot|Cerebellum|Subcortical") & corr_mean_df["ALL"]['seed2'].str.contains("SomMot|Cerebellum|Subcortical")].reset_index(drop=True)
df_half_SomMot=corr_half_concat_df["ALL"][corr_half_concat_df["ALL"]['seed1'].str.contains("SomMot|Cerebellum|Subcortical") & corr_half_concat_df["ALL"]['seed2'].str.contains("SomMot|Cerebellum|Subcortical")].reset_index(drop=True)

SomMot_list = [label for label in labels_list if re.search(r"SomMot|Cerebellum|Subcortical", label)]
networks_lat_list = [('L' if 'LH' in x.split('_')[0] else 'R') + ' ' + x.split('_')[1] for x in labels_list] # list of the 17 networks corresponding to each roi
networks_list = [x.split('_')[1] for x in labels_list] # list of the 17 networks corresponding to each roi

network_SomMot_list = [('L' if 'LH' in x.split('_')[0] else 'R') + ' ' + x.split('_')[1] for x in SomMot_list] # list of the R or L network name for each roi


#### <font color=#04AF98> A.3 Plot the sub-matrix (SomMot)

In [ ]:
plot.plot_heatmap(matrix=df_SomMot,
                  networks=network_SomMot_list,
                      networks_y=network_SomMot_list,
                      vmax=0.4,vmin=-0.05,
                      cmap=fc_cmap,#half="upper",
                      index_raw='seed1',index_col='seed2',metric='fcorr',
                  xlabels=True,ylabels=True,
                       output_f=output_dir + '/figures/FC_matrix_brain.pdf',
                      save=False)
plt.show()

#### <font color=#04AF98> A.3 Intra vs Inter FC

In [ ]:
# Load spinal cord data
mean_betwith_sc = pd.read_csv(glob.glob(output_dir.split("brain")[0] + 'spinalcord/2_second_level/n67_betwith_fcorr.csv')[0])

# Plot params
colors = ["#0C675F","#3AA198","#68BCB4","#86C9C3","#C3E4E1","#F1C2CF","#E3849F","#D5476F","#C7093F","#850429"]  # blue → white → red
custom_cmap = LinearSegmentedColormap.from_list("my_colormap", colors)
limits={"intra":[-0.1,0.5],"inter":[-0.05,0.15]}

#----- Compute mean intra or inter network fcorr value and create the related df
mean_betwith_SomMot={};all_fcorr={}

for i in ["intra","inter"]:
    
    for net_nb, network in enumerate(np.append(np.unique(networks_list), "SpinalCord")):
        
        mean_network=corr_half_concat_df["ALL"][corr_half_concat_df["ALL"]['seed1'].str.contains(network) | corr_half_concat_df["ALL"]['seed2'].str.contains(network)].reset_index(drop=True)
        mean_betwith_SomMot[network] =mean_network.groupby(["IDs","age","sex","betwith_labels"], sort=False)[["fcorr"]].mean().reset_index()

        if network=="SpinalCord":
            mean_betwith_SomMot["SpinalCord"] = mean_betwith_sc.copy()
        mean_betwith_SomMot[network] =mean_betwith_SomMot[network][mean_betwith_SomMot[network]["betwith_labels"]==i]

        mean_betwith_SomMot[network]["network"]=network
            
        output_f=output_dir + '2_second_level/n67_'+network+'_betwith_fcorr.csv'
        if not os.path.exists(output_f):
            mean_betwith_SomMot[network].to_csv(output_f, index=False)

        if net_nb==0:
            all_fcorr[i]=mean_betwith_SomMot[network]
        else:
            all_fcorr[i]=pd.concat((all_fcorr[i],mean_betwith_SomMot[network]))


    plot.boxplots(df=all_fcorr[i],
                      x_data="network",x_order=['SpinalCord','Subcortical','Cerebellum','SomMot','Vis','SalVentAttn','Limbic','DorsAttn','Default',  'Cont'],
                          indiv_values=True,#invers_axes=True,
                          indiv_hue="age",indiv_color=custom_cmap,
                          palette=["gray","gray","gray","gray","gray","gray","gray","gray","gray","gray"],#output_dir=config['main_dir'] + config['analysis_dir']['spinalcord'] + '/figures/',
                          #output_tag='corr_' + group_name,
                      height=5,aspect=1,
                          y_data="fcorr",
                      ymin=limits["intra"][0], ymax=limits["intra"][1],
                      output_dir=output_dir + "/figures/",
                      output_tag="fcorr_"+i+"_allnetworks" ,plot_legend=False,
                      save=False)
    plt.show()

### Stats: intra vs inter

In [ ]:
from scipy.stats import ttest_rel, wilcoxon, ttest_1samp


#--------- global statistics across networks
intra_mean = all_fcorr["intra"].groupby("IDs")["fcorr"].mean().rename("fcorr_intra")
inter_mean = all_fcorr["inter"].groupby("IDs")["fcorr"].mean().rename("fcorr_inter")
paired_df = pd.concat([intra_mean, inter_mean], axis=1).dropna()
t_stat, p_val = ttest_rel(paired_df["fcorr_intra"], paired_df["fcorr_inter"])
print("global results: " + str(np.round(t_stat,2)) + " "+ str(p_val))
print(str(np.round(np.mean(intra_mean),2)) + " "+ str(np.round(np.std(intra_mean),2)))
print(str(np.round(np.mean(inter_mean),2)) + " "+ str(np.round(np.std(inter_mean),2)))

#--------- statistics for each networks
merged = pd.merge(all_fcorr["intra"], all_fcorr["inter"], on=["IDs", "network"], suffixes=('_intra', '_inter'))

results = []
for network in merged["network"].unique():
    df_sub = merged[merged["network"] == network]
    t_stat, p_val = ttest_rel(df_sub["fcorr_intra"], df_sub["fcorr_inter"])
    results.append({"network": network,"mean_intra": df_sub["fcorr_intra"].mean(),"std_intra": df_sub["fcorr_intra"].std()
                    ,"mean_inter": df_sub["fcorr_inter"].mean(),"std_inter": df_sub["fcorr_inter"].std(),
        "t_stat": t_stat,"p_val": p_val})
print(" ")
print("results by networks")
results_df = pd.DataFrame(results)
print(results_df)


### Stats: intra or inter versus 0

In [ ]:
#--------- global statistics across networks
t_stat, p_val = ttest_1samp(paired_df["fcorr_inter"], 0) 
print("global results: " + str(np.round(t_stat,2)) + " "+ str(p_val))

#--------- statistics for each networks
merged = pd.merge(all_fcorr["intra"], all_fcorr["inter"], on=["IDs", "network"], suffixes=('_intra', '_inter'))

results = []
for network in merged["network"].unique():
    df_sub = merged[merged["network"] == network]
    t_stat, p_val = ttest_1samp(df_sub["fcorr_inter"], 0)
    results.append({"network": network,"t_stat": t_stat,"p_val": p_val})
print(" ")
print("results by networks")
results_df = pd.DataFrame(results)
print(results_df)

<hr style="border:1px solid black">

### <font color=#04AF98> B. Age-effect

In [ ]:
output_dir_table=config["project_dir"] +  "/figures/f02_functional_sc/revision_R1/seed2seed/brain/"
# Initialize result containers
roi_results = {}
all_results_df = {}

# Loop over 'intra' and 'inter' ROI pairs
all_betwith=pd.concat((all_fcorr["intra"],all_fcorr["inter"]),axis=0)
for roi_nb, roi in enumerate(["intra", "inter"]):
    results=[]
    roi_results[roi] = []
    all_results_df[roi] = pd.DataFrame(columns=["Network", "ROI","Predictor","β","95% Bootstrap CI","t-value","p-value","p-fdr"])
    
    if not os.path.exists(output_dir_table + "stats_brain_"+roi+"_by_network.csv"):
        # Analyze each network
        for net_nb, network in enumerate(np.append(np.unique(networks_list), "SpinalCord")):  #
            sub_df = all_betwith[all_betwith["network"]==network]
            sub_df = sub_df[sub_df["betwith_labels"] == roi].copy()
        
            # Compute statistics
            signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(
                    df=sub_df, y="fcorr", predictor="age", covariates=["sex"],n_bootstrap=1000)
                
            results.append({"Network": network, "ROI":roi,"Predictor": "age","β": beta_age,"95% Bootstrap CI": ci_beta_age,"t-value": stat_age,"p-value": p_age,"p-fdr": p_age,})
            results.append({"Network": network,"ROI":roi,"Predictor": "sex","β": beta_sex,"95% Bootstrap CI": ci_beta_sex,"t-value": stat_sex,"p-value": p_sex,"p-fdr": p_sex,})
            
        roi_results_df = pd.DataFrame(results)
        
        all_results_df[roi] = pd.concat([all_results_df[roi], roi_results_df])
        
        # FDR correction
        for predictor in all_results_df[roi]["Predictor"].unique():
            mask = all_results_df[roi]["Predictor"] == predictor
            pvals = all_results_df[roi].loc[mask, "p-value"].values
    
            _, pvals_fdr, _, _ = multipletests(pvals,alpha=0.05,method="fdr_bh")
    
            all_results_df[roi].loc[mask, "p-fdr"] = pvals_fdr
        all_results_df[roi].to_csv(output_dir_table + "stats_brain_"+roi+"_by_network.csv", index=False)

    
    else:
        all_results_df[roi] =pd.read_csv(output_dir_table + "stats_brain_"+roi+"_by_network.csv")



In [ ]:
# Add ROI type as a new column and concatenate
df_plot = pd.concat([
    df.assign(roi=roi_type) for roi_type, df in all_results_df.items()
])
df_plot=df_plot[df_plot["Predictor"]=="age"]
df_plot = df_plot.sort_values(by=["Network", "ROI"])

color=["#E2971F","#e7cda4"]

plot.barplots(df=df_plot, x_data="Network",x_order=['SpinalCord','Subcortical','Cerebellum','SomMot','Vis','SalVentAttn','Limbic','DorsAttn','Default',  'Cont'],
                  y_data="t-value", hue="roi",
                  palette=color,
                  ymin=-1,ymax=6,indiv_values=False,
                output_dir=output_dir + "/figures/" ,
                  height=3,aspect=1,
                  output_tag="age_effect_fcorr",save=False)


In [ ]:
# FDR correction
df_all["p-fdr"] = None  # initialize column

if not os.path.exists(output_dir_table + "stats_sc_all_by_horn.csv"):
    for predictor in df_all["Predictor"].unique():
        mask = df_all["Predictor"] == predictor
        pvals = df_all.loc[mask, "p-value"].values
    
        _, pvals_fdr, _, _ = multipletests(pvals,alpha=0.05,method="fdr_bh")
    
        df_all.loc[mask, "p-fdr"] = pvals_fdr
    df_all.to_csv(output_dir_table + "stats_sc_all_by_horn.csv", index=False)
else:
    df_all = pd.read_csv(output_dir_table + "stats_sc_all_by_horn.csv")
#df_all
for roi in np.unique(df_all["ROI"]):
    print(roi + " absolute mean t-value: " + str(np.round(np.mean(np.abs(df_all[(df_all["ROI"]==roi)
                  &(df_all["Predictor"]=='age')]["t-value"])),3)) + " ± " +
         str(np.round(np.std(np.abs(df_all[(df_all["ROI"]==roi)
                  &(df_all["Predictor"]=='age')]["t-value"])),2)))

In [ ]:
pvals = all_results_df[roi]["p_age"].values
    _, pvals_fdr, _, _ = multipletests(pvals, method='fdr_bh')
    all_results_df[roi]["p_age_fdr"]=pvals_fdr

In [ ]:

# Initialize result containers
roi_results = {}
all_results_df = {}

# Loop over 'intra' and 'inter' ROI pairs
all_betwith=pd.concat((all_fcorr["intra"],all_fcorr["inter"]),axis=0)
for roi_nb, roi in enumerate(["intra", "inter"]):
    results=[]
    all_results_df[roi] = pd.DataFrame(columns=[
        "network", "signed_r2", "p_age", "p_sex",
        "beta_age", "beta_sex", "tvalue_age", "tvalue_sex"
    ])
    
    roi_results[roi] = []

    # Analyze each network
    for net_nb, network in enumerate(np.append(np.unique(networks_list), "SpinalCord")):  #
        sub_df = all_betwith[all_betwith["network"]==network]
        sub_df = sub_df[sub_df["betwith_labels"] == roi].copy()

        # Compute statistics
        signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(
            df=sub_df, y="fcorr", predictor="age", covariates=["sex"],n_bootstrap=1000)
        
        results.append({"Network": network,"Predictor": "age","β": beta_age,"95% Bootstrap CI": ci_beta_age,"t-value": stat_age,"p-value": p_age,"p-fdr": p_age,})
        results.append({"Network": network,"Predictor": "sex","β": beta_sex,"95% Bootstrap CI": ci_beta_sex,"t-value": stat_sex,"p-value": p_sex,"p-fdr": p_sex,})
    
    roi_results[roi].append(results)

    roi_results_df = pd.DataFrame(roi_results[roi])
    all_results_df[roi] = pd.concat([all_results_df[roi], roi_results_df])
    pvals = all_results_df[roi]["p_age"].values
    _, pvals_fdr, _, _ = multipletests(pvals, method='fdr_bh')
    all_results_df[roi]["p_age_fdr"]=pvals_fdr

# Add ROI type as a new column and concatenate
df_plot = pd.concat([
    df.assign(roi=roi_type) for roi_type, df in all_results_df.items()
])
df_plot = df_plot.sort_values(by=["network", "roi"])
color=["#E2971F","#e7cda4"]

plot.barplots(df=df_plot, x_data="network",x_order=['SpinalCord','Subcortical','Cerebellum','SomMot','Vis','SalVentAttn','Limbic','DorsAttn','Default',  'Cont'],
                  y_data="tvalue_age", hue="roi",
                  palette=color,
                  ymin=-1,ymax=6,indiv_values=False,
                output_dir=output_dir + "/figures/" ,
                  height=3,aspect=1,
                  output_tag="age_effect_fcorr",save=False)



In [ ]:
output_dir_table=config["project_dir"] +  "/figures/f02_functional_sc/revision_R1/seed2seed/brain/"

# Initialize result containers
roi_results = {}
all_results_df = {}

# Loop over 'intra' and 'inter' ROI pairs
all_betwith=pd.concat((all_fcorr["intra"],all_fcorr["inter"]),axis=0)
for roi_nb, roi in enumerate(["intra", "inter"]):
    all_results_df[roi] = pd.DataFrame(columns=[
        "network", "signed_r2", "p_age", "p_sex",
        "beta_age", "beta_sex", "tvalue_age", "tvalue_sex"
    ])
    
    roi_results[roi] = []

    # Analyze each network
    for net_nb, network in enumerate(np.append(np.unique(networks_list), "SpinalCord")):  #
        sub_df = all_betwith[all_betwith["network"]==network]
        sub_df = sub_df[sub_df["betwith_labels"] == roi].copy()

         # Compute statistics
        signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(
            df=sub_df, y="fcorr", predictor="age", covariates=["sex"])
        

        result = {"network": network,"signed_r2": signed_r2,
                  "p_age": p_age,"p_sex": p_sex,
                  "beta_age": beta_age,"beta_sex": beta_sex,
                  "tvalue_age": stat_age,"tvalue_sex": stat_sex}

        roi_results[roi].append(result)

        #------- Plotting
        #plot.lmplots(
         #    df=sub_df,
          #  color=["grey"],hue_color_var="age",hue_palette=custom_cmap,
           #  x_data="age",y_data="fcorr",
            # xmin=15,xmax=85,
             #indiv_values=True,save=False)
   


        
    roi_results_df = pd.DataFrame(roi_results[roi])
    all_results_df[roi] = pd.concat([all_results_df[roi], roi_results_df])
    pvals = all_results_df[roi]["p_age"].values
    _, pvals_fdr, _, _ = multipletests(pvals, method='fdr_bh')
    all_results_df[roi]["p_age_fdr"]=pvals_fdr

# Add ROI type as a new column and concatenate
df_plot = pd.concat([
    df.assign(roi=roi_type) for roi_type, df in all_results_df.items()
])
df_plot = df_plot.sort_values(by=["network", "roi"])
color=["#E2971F","#e7cda4"]

plot.barplots(df=df_plot, x_data="network",x_order=['SpinalCord','Subcortical','Cerebellum','SomMot','Vis','SalVentAttn','Limbic','DorsAttn','Default',  'Cont'],
                  y_data="tvalue_age", hue="roi",
                  palette=color,
                  ymin=-1,ymax=6,indiv_values=False,
                output_dir=output_dir + "/figures/" ,
                  height=3,aspect=1,
                  output_tag="age_effect_fcorr",save=False)


In [ ]:
all_results_df["inter"]

In [ ]:
all_results_df["intra"]